In [26]:
import cv2
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision


In [42]:
img = cv2.imread("media/fotos/1t.jpg")
if img is None:
    print('No hay foto ')
else:
    cv2.imshow('Image', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [ ]:
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np

MARGIN = 10  # pixels
FONT_SIZE = 1
FONT_THICKNESS = 1
HANDEDNESS_TEXT_COLOR = (88, 205, 54) # vibrant green

def draw_landmarks_on_image(rgb_image, detection_result):
  hand_landmarks_list = detection_result.hand_landmarks
  handedness_list = detection_result.handedness
  annotated_image = np.copy(rgb_image)

  # Loop through the detected hands to visualize.
  for idx in range(len(hand_landmarks_list)):
    hand_landmarks = hand_landmarks_list[idx]
    handedness = handedness_list[idx]

    # Draw the hand landmarks.
    hand_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    hand_landmarks_proto.landmark.extend([
      landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in hand_landmarks
    ])
    solutions.drawing_utils.draw_landmarks(
      annotated_image,
      hand_landmarks_proto,
      solutions.hands.HAND_CONNECTIONS,
      solutions.drawing_styles.get_default_hand_landmarks_style(),
      solutions.drawing_styles.get_default_hand_connections_style())

    # Get the top left corner of the detected hand's bounding box.
    height, width, _ = annotated_image.shape
    x_coordinates = [landmark.x for landmark in hand_landmarks]
    y_coordinates = [landmark.y for landmark in hand_landmarks]
    text_x = int(min(x_coordinates) * width)
    text_y = int(min(y_coordinates) * height) - MARGIN

    # Draw handedness (left or right hand) on the image.
    cv2.putText(annotated_image, f"{handedness[0].category_name}",
                (text_x, text_y), cv2.FONT_HERSHEY_DUPLEX,
                FONT_SIZE, HANDEDNESS_TEXT_COLOR, FONT_THICKNESS, cv2.LINE_AA)

  return annotated_image

In [40]:
mp_image = mp.Image.create_from_file('media/fotos/1t.jpg')
model_path = 'hand_landmarker.task'

BaseOptions = mp.tasks.BaseOptions
HandLandmarker = mp.tasks.vision.HandLandmarker
HandLandmarkerOptions = mp.tasks.vision.HandLandmarkerOptions
VisionRunningMode = mp.tasks.vision.RunningMode

# Create a hand landmarker instance with the image mode:
options = HandLandmarkerOptions(
    base_options=BaseOptions(model_asset_path='hand_landmarker.task'),
    running_mode=VisionRunningMode.IMAGE)

In [50]:

with HandLandmarker.create_from_options(options) as landmarker:
  # The landmarker is initialized. Use it here.
  # ...
    hand_landmarker_result = landmarker.detect(mp_image)
    
    landmarks = hand_landmarker_result.hand_landmarks
    if landmarks:
      for hand_landmarks in landmarks:
          thumb_tip = hand_landmarks[4]  # Índice 4 para la punta del pulgar
          index_tip = hand_landmarks[8]  # Índice 8 para la punta del índice

          thumb_x, thumb_y, thumb_z = thumb_tip.x, thumb_tip.y, thumb_tip.z
          index_x, index_y, index_z = index_tip.x, index_tip.y, index_tip.z

          # Calcular la distancia entre la punta del pulgar y la del índice
          distance = ((thumb_x - index_x) ** 2 + (thumb_y - index_y) ** 2 + (thumb_z - index_z) ** 2) ** 0.5

          # Establecer un umbral para considerar que están juntos
          threshold = 0.06

          if distance < threshold:
              print("La punta del pulgar y la del índice están juntas.")
          else:
              print("La punta del pulgar y la del índice están separadas.")
          print(hand_landmarks[0])


    """
    annotated_image = draw_landmarks_on_image(mp_image.numpy_view(), hand_landmarker_result)


    cv2.imshow('Image', cv2.cvtColor(annotated_image, cv2.COLOR_RGB2BGR))
    cv2.waitKey(0)
    cv2.destroyAllWindows()
  """
    

La punta del pulgar y la del índice están juntas.
NormalizedLandmark(x=0.7964830994606018, y=0.7010098695755005, z=-5.079517251260768e-08, visibility=0.0, presence=0.0)


In [2]:

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    cv2.imshow('Video', frame)

    

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()